In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from utils.MIND import MIND_iter,MIND_map
from utils.utils import constructBasicDict

## show data

In [13]:
hparams = {
    'mode':'demo',
    'batch_size':2,
    'his_size':2,
    'title_size':18,
    'gpu':'cuda:0',
    'attrs': ['title','category','subcategory']
}

# customize your path here

news_file_train = 'D:/Data/NR_data/dev/news_train.tsv'
news_file_test = 'D:/Data/NR_data/dev/news_test.tsv'
behavior_file_train = 'D:/Data/NR_data/dev/behaviors_train.tsv'
behavior_file_test = 'D:/Data/NR_data/dev/behaviors_test.tsv'

# if user2id,word2id,news2id haven't been constructed
if not os.path.exists('data/nid2idx_{}_{}.json'.format(hparams['mode'],'train')):
    constructBasicDict(news_file_train,behavior_file_train,hparams['mode'],'train',hparams['attrs'])

if not os.path.exists('data/nid2idx_{}_{}.json'.format(hparams['mode'],'test')):
    constructBasicDict(news_file_test,behavior_file_test,hparams['mode'],'test',hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

# map style dataloader to return one log for training
dataset_train = MIND_map(hparams=hparams,mode='train',npratio=4,news_file=news_file_train,behaviors_file=behavior_file_train)

# iter style dataloader to return one candidate for evaluating
dataset_test = MIND_iter(hparams=hparams,mode='test',news_file=news_file_test,behaviors_file=behavior_file_test)

vocab_train = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab_train.load_vectors(embedding)

vocab_test = dataset_test.vocab
vocab_test.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [14]:
a = next(iter(loader_train))
b = next(iter(loader_test))

In [15]:
a,b

({'user_index': tensor([[ 727],
          [1389]], dtype=torch.int32),
  'clicked_title': tensor([[[ 6262,  4554,     6,    71,    30,    40,  5749,  3524, 26295,  3525,
               21,     8,   482,     0,     0,     0,     0,     0],
           [  814,   906,  2632,    65,    36,    74,  3988,  2892,   716,  1135,
                9,  5183,    14,   948,     0,     0,     0,     0]],
  
          [[    8,  2051,  4245,     9,  4381,  2441,     5,  2892,  8924,  5667,
              519,     0,     0,     0,     0,     0,     0,     0],
           [  317,    71,   192,  5430,    47,   144,    80,   845,    10,   331,
                0,     0,     0,     0,     0,     0,     0,     0]]],
         dtype=torch.int32),
  'clicked_category': tensor([[[26],
           [24]],
  
          [[26],
           [ 3]]], dtype=torch.int32),
  'clicked_subcategory': tensor([[[122],
           [343]],
  
          [[321],
           [ 13]]], dtype=torch.int32),
  'candidate_title': tensor([[[  814, 

### tailor Data to demo size

In [ ]:
tailorData('D:/Data/NR_data/MINDsmall_dev/behaviors.tsv',300)
tailorData('D:/Data/NR_data/MINDsmall_train/behaviors.tsv',300)